# BCB Final Project - a.y. 17/18

### <i>Fiaschi Lorenzo, Franco Danilo</i>

<ol> <b>TODO</b>
    <li><strike>problem presentation (dataset, objectives)</strike></li>
    <li><strike>preprocessing</strike></li>
    <li><strike>mkl procedure overview</strike></li>
    <li><strike>chosen model description (alignment model, cortes approach)</strike></li>
    <li>algorithm pipeline (with a major point in the cross-validation decisions, how it is shuffled...)</li>
    <li>approach settings</li>
    <li>performances over toy (with description on how it is generated</li>
    <li>integration needed? comparison with single dataset model</li>
    <li>performances over real dataset</li>
    <li>what we can learn from the best configuration (sparsity)</li>
    <li>kernel integration on the only meaningful dataset (clinical)</li>
    <li>overview on the regression approach</li>
    <li>regression performances with data integration</li>
    <li>single kernel approach after regression results</li>
    <li>final comments over the results</li>
</ol>

### 1. The Problem
The dataset consists of 2741 aligned patients affected by Diabetes; it is structured in 3 differents tables:
<ol>
    <li>Genetic Features, 347 - Related to the SNP of 344 genes + 3 composite genes scores [blood pressure, alzheimer, CVD]
    </li><li>Retina's Features, 157 - Engineered features extracted from patients' retinas</li>
    <li>Clinical Features, 15 - General patients' information</li>
</ol>
The objective of the study is to find a model that is able to predict whether a patient will incurr in either cardiovascular failures or in episodes of dementia.<br>
In the positive case, it is also aked to predict at which age this episodes will occurr.<br>
In order to learn such model, the output dataset has been provided; it is composed by two information, both for the occurrence of a cardiovascular failure and for the dementia episode: whether the episode has happened or not and at what age.

### 2. Data Preprocessing
In order to be fed, the tables have been cleaned as follows:
<ul>
    <li>
        <b><i>NaN Filling</i></b>: 
        <ul>
            <li>
                Clinical DS: 
                <ul>
                    <li>
                        ["therapy","gender","precedent CVD", "smoker"] $\rightarrow$ most frequent;
                    </li>
                    <li>
                        Others $\rightarrow$ mean.
                    </li>
                </ul>
            <li>
                Genetic DS: 
                <ul>
                    <li>
                        [composite gene scores] $\rightarrow$ mean;
                    </li>
                    <li>
                        Others $\rightarrow$ min.
                    </li>
                </ul>
            <li>
                Vampire DS: 
                <ul>
                    <li>
                        All $\rightarrow$ mean.
                    </li>
                </ul>
        </ul>
    </li>
    <li>
        <b><i>One-Hot Encoding</i></b>:
        <ul>
            <li>Clinical DS:
                <ul>
                    <li>
                        "therapy" and "Apoe4Presence".
                    </li>
                </ul>
            </li>
        </ul>
    </li>
    <li><b><i>Boolean: from binary to symmetric</i></b>: 
        <ul>
            <li>Clinical DS:
                <ul>
                    <li>
                        ["gender","precedent CVD", "smoker"] $\rightarrow$ from [0,1] to [-1,1].
                    </li>
                </ul>
            </li>
            <li>Outputs:
                <ul>
                    <li>
                        ["cvd_fail","dement_fail"] $\rightarrow$ from [0,1] to [-1,1].
                    </li>
                </ul>
            </li>
        </ul>
    </li>
</ul>

### 3. Multiple Kernel Learning
<b>Multiple Kernel Learning</b> refers to a set of machine learning methods that use a predefined set of kernels and learn an optimal linear or non-linear combination of kernels as part of the algorithm.<br>
Reasons to use multiple kernel learning include:
<ul>
    <li>the ability to select for an optimal kernel and parameters from a larger set of kernels, reducing bias due to kernel selection while allowing for more automated machine learning methods;
    </li>
    <li>combining data from different sources (e.g. sound and images from a video) that have different notions of similarity and thus require different kernels;
    </li>
</ul>
<img src='others/mkl.png'>

In [1]:
%%javascript
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } }
});

<IPython.core.display.Javascript object>

### 4. The Model
<ul>
    <li><b><i>Target Function</i></b> $\rightarrow$ Similarity-based function; it uses a similarity metric between the combined kernel matrix and an optimum kernel matrix calculated from the training data, in order to select the combination function parameters that maximize the similarity. The similarity between two kernel matrices can be calculated using kernel alignment, Euclidean distance, Kullback-Leibler (KL) divergence, or any other similarity measure.
    </li>
    <li><b><i>Training Method</i></b> $\rightarrow$ One-step method; it calculates both the combination function parameters and the parameters of the combined base learner in a single pass. One can use a sequential approach or a simultaneous approach. In the sequential approach, the combination function parameters are determined first, and then a kernel-based learner is trained using the combined kernel. In the simultaneous approach, both set of parameters are learned together.
    </li>
</ul>
The chosen algorithm is called <i>Centered-kernel alignment</i> [1] and its main purpose is to compute a set of $P$ kernels $K_i$, $i \, = \, 1...P$ from the datasets and to use a linear combination of them to approximate an ideal kernel. Then the obtained approximated kernel can be used for both classification and regression purposes. The functional is:<br>

\begin{equation}
\max\limits_{\eta \in \mathcal{M}} CA(K_{\eta}, IK)
\label{eq:problem}
\end{equation}

where $IK = y^T y$ is the ideal kernel, $K_{\eta} = \sum\limits_{i=1}^P K_i \eta_i$ is its approximation and $\mathcal{M}=\{\eta : ||\eta||_2 = 1\}$ imposing $\eta$ being a unit norm vector. In turn, $CA(K_1, K_2)$ is the defined as

\begin{equation*}
CA(K_1^c, K_2^c) = \frac{\langle K_1^c, K_2^c \rangle_F}{\sqrt{\langle K_1^c, K_1^c\rangle_F \; \langle K_2^c, K_2^c \rangle_F}}
\end{equation*}

with $K^c$ is the centered version of $K$ and can be calculated as

\begin{equation*}
K^c = K - \frac{1}{N} 11^TK - \frac{1}{N} K11^T + \frac{1}{N^2} \left(1^TK1\right)11^T
\end{equation*}

where $1$ is the vector of ones with proper dimension.

The optimization problem \eqref{eq:problem} has then a unique analytical solution

\begin{equation}
\eta = \frac{M^{-1}a}{||M^{-1}a||_2}
\label{eq:problem_solution}
\end{equation}

where $M = \{\langle K_m, K_h\rangle_F\}_{m,h \, = \, 1}^P$ and $a = \{\langle K_m, IK\rangle_F\}_{m \, = \, 1}^P$.

### 5. The Algorithm

<ol>
    <li><i>Datasets loading</i> $\rightarrow$ (Clinical, Genetic, Vampire, Outputs);</li>
    <li><i>Outer split</i> $\rightarrow$ 75% training + 25% test for final testing purposes;</li>
    <li><i>Kernel definition</i> $\rightarrow$ dictionary list: $[\{\;<kernelType1>:[parameter list],\;<kernelType2>:[parameter list],\;\cdots\;\},\;\cdots\;]$</li>
    <li><i>Sampling rounds</i> $\rightarrow$ 3 rounds of 75% training + 25% mid test (extracted from the previous 75% of training) for statistical stability:
        <ol type="i">
            <li></li>
        </ol>
    </li>
</ol>

(the sampling results will be processed in order to find the best configuration over one dictionary):

### References
[1] Corinna Cortes, Mehryar Mohri, and Rostamizadeh Afshin. Two-stage learning kernel algorithms. In Proceedings of the 27th International Conference on Machine Learning, 2010a.